# Tutorial 3: The Numerical Sandbox, Extension Experiment (Static mesh version)

**Romain Beucher**
Version 0.1

romain.beucher@unimelb.edu.au

The following notebook is an implementation of the Numerical Sandbox Extension Experiment similar to [Buiter et al., 2006](http://sp.lyellcollection.org/content/253/1/29). The test is commonly referred as one of the GEOMOD benchmarks and is used to test the large deformation viscous-plastic behaviour of geodynamic numerical codes.

The initial purpose of the GEOMOD numerical experiments was to compare results from analog and numerical experiments. It is not a numerical benchmark sensus-stricto.

The test has been implemented using Fantom in [Thieulot, 2011](http://linkinghub.elsevier.com/retrieve/pii/S0031920111001336) and more recently with Aspect in [Glerum et al., 2017](http://www.solid-earth-discuss.net/se-2017-9). 

![title](images/underworld.png)

Results for Underworld from this notebook

A | B
- | - 
![title](images/ExtensionA.png) | ![title](images/ExtensionB.png) 

Results for the extension experiment after 2cm extension (Buiter et al., 2006)

In [ ]:
import UWGeodynamics as GEO
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Image
from IPython.display import clear_output

In [ ]:
u = GEO.UnitRegistry

# Model Geometry

Set-up of the extension experiment. A viscous layer (PDMS, 10 x 0.5 cm) lies in the central part of the model on the base. The rest of the model consists of three ‘sand’ layers (only differing in colour). Extension is achieved by moving the right wall with the attached 10 cm long sheet outwards to the right.

<img src="images/Extension.png" width=800>

We use a uniform resolution of 0.2 cm (400 x 100 elements)

In [ ]:
outputDir="outputs_tutorial3"

# define all widgets

fs = {'description_width': 'initial'} # full style
time = widgets.IntSlider(description='no. minutes ', min=1,max=60,style=fs, value=2)
wall_vx = widgets.BoundedFloatText(value=2.5,min=-200.,max=200.0,
                                   description='Right wall velocity [cm/hr]', style=fs,disabled=False )
res = widgets.BoundedFloatText(value=0.1,min=0.02,max=1.0,
                               description='Initial Resolution [cm]', style=fs,disabled=False )
outputbox = widgets.Text(
    value='output/extension',
    description='output path:',
    disabled=False
)

w_button = widgets.Button(
    description='Do science',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

outputPath = './outputs_tutorial3'

def display_options():
    display(Image('../images/Extension.png'))
    display(time)
    display(wall_vx)
    display(res)
    display(outputbox)
    display(w_button)

In [ ]:
velocity = wall_vx.value * u.centimeter / u.hour
model_length = 20. * u.centimeter
bodyforce = 1560 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / velocity
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [ ]:
def make_model(d):
    
#     display_options()
    
    global outputPath 
    outputPath = str(outputbox.value)
    import os
    outputPath = os.path.abspath(outputPath)+'/'
    
    import math
    nx = model_length.magnitude / res.value
    ny = 5. / res.value
    
    nx = int(math.ceil(nx / 2.) * 2)
    ny = int(math.ceil(ny / 2.) * 2)
    
    Model = GEO.Model(elementRes=(nx, ny), 
                      minCoord=(0. * u.centimeter, -3.5 * u.centimeter), 
                      maxCoord=(model_length, 1.5 * u.centimeter), 
                      gravity=(0.0, -9.81 * u.meter / u.second**2))
    print("Running with {} x {} elements".format(nx, ny))
    Model.outputDir = outputPath
    Model.add_visugrid(elementRes=(100, 25), 
                   minCoord=(0. * u.centimeter, -3.5 * u.centimeter), 
                   maxCoord=(model_length, 0.0 * u.centimeter))

    air = Model.add_material(name="Air", shape=GEO.shapes.Layer(top=Model.top, bottom=Model.bottom))
    sand1 = Model.add_material(name="Sand1", shape=GEO.shapes.Layer(top=0.*u.centimeter, bottom=Model.bottom))
    sand2 = Model.add_material(name="Sand2", shape=GEO.shapes.Layer(top=-1. * u.centimeter, bottom=-2. * u.centimeter))

    vertices = [( 5.* u.centimeter, -3.0 * u.centimeter),
                (15.* u.centimeter, -3.0 * u.centimeter),
                (15.* u.centimeter, -3.5 * u.centimeter),
                ( 5.* u.centimeter, -3.5 * u.centimeter)]

    silicon = Model.add_material(name="Silicon", shape=GEO.shapes.Polygon(vertices))
    
    air.density = 10. * u.kilogram / u.metre**3
    sand1.density = 1560. * u.kilogram / u.metre**3
    sand2.density = 1560. * u.kilogram / u.metre**3
    silicon.density = 965. * u.kilogram / u.metre**3
    
    # Assign function to materials
    air.viscosity     = GEO.ConstantViscosity(1.0e2  * u.pascal * u.second)
    sand1.viscosity   = GEO.ConstantViscosity(1.0e13 * u.pascal * u.second)
    sand2.viscosity   = GEO.ConstantViscosity(1.0e13 * u.pascal * u.second)
    silicon.viscosity = GEO.ConstantViscosity(5.0e4  * u.pascal * u.second)

    sandPlasticity = GEO.DruckerPrager(cohesion=10. * u.pascal,
                                         cohesionAfterSoftening=10. * u.pascal,
                                         frictionCoefficient=0.73,
                                         frictionAfterSoftening=0.60,
                                         minimumViscosity=1.0e2  * u.pascal * u.second)
    
    sand1.plasticity = sandPlasticity
    sand2.plasticity = sandPlasticity
    
    conditions = [(Model.x > GEO.nd(10.1 * u.centimetre), GEO.nd(velocity)),
              (Model.x > GEO.nd(9.9 * u.centimetre), (Model.x - GEO.nd(9.9 * u.centimetre)) * GEO.nd(velocity) / GEO.nd(0.2 * u.centimetre)),
              (True, 0.0)]

    Model.set_velocityBCs(left=[0 * u.centimeter / u.hour, None],
                          right=[velocity, None],
                          bottom=[conditions, 0.0])
    
    Model.minViscosity = 1.0e2 * u.pascal * u.second
    Model.maxViscosity = 1.0e9 * u.pascal * u.second
    Model.init_model()
    
    GEO.rcParams["solver"] = "mumps"
    GEO.rcParams["penalty"] = 1e6

    Model.run_for(time.value * u.minutes, checkpoint_interval=3 * u.minutes, glucifer_outputs=True)

In [ ]:
w_button.on_click(make_model)
display_options()

In [ ]:
def vizualise(path=None):
    # if no path provided use the global value from the widget
    if path == None:
        global outputPath
        path = outputPath
    import os
    if not os.path.exists(path):
        print("Can't find ", path)
        return

    vis = glucifer.lavavu.Viewer(database=path+'/glucifer.gldb')
    vis.control.Panel()

    vis.control.TimeStepper()
    vis.control.ObjectList()
    vis.control.show()